<a href="https://colab.research.google.com/github/JairsonAlbertoSami/desafio-junior-data-scientis/blob/main/visualizacao_Analise_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instala BigQuery e basedosdados

!pip install google-cloud-bigquery
!pip install basedosdados